In [1]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from tensorflow import keras
import tensorflow as tf

In [3]:
# Simulated battery data
'''
data = [
    {"batteryId": 1, "stateOfCharge": 0.4384, "stateOfHealth": 98.00, "batteryCurrent": 349.57, "batteryVoltage": 397.63,
     "kmh": 234.00, "distance": 79.35, "batteryTemp": 0.00, "ambientTemp": 20.40},
    {"batteryId": 2, "stateOfCharge": 0.1234, "stateOfHealth": 95.00, "batteryCurrent": 500.00, "batteryVoltage": 350.00,
     "kmh": 200.00, "distance": 100.50, "batteryTemp": 60.00, "ambientTemp": 25.40},
    {"batteryId": 3, "stateOfCharge": 0.7684, "stateOfHealth": 99.00, "batteryCurrent": 200.00, "batteryVoltage": 400.00,
     "kmh": 180.00, "distance": 120.00, "batteryTemp": 30.00, "ambientTemp": 20.00},
]
'''

# Read the data from file
df = pd.read_csv("./data/battery_data.csv")

print(df)

                            timestamp  batteryId  ambientTemp  batteryCurrent  \
0    2025-02-12 14:26:34.190000+00:00          1        18.65           78.06   
1    2025-02-12 14:26:37.025000+00:00          1        18.36           81.42   
2    2025-02-12 14:26:40.020000+00:00          1        18.17           75.76   
3    2025-02-12 14:26:43.024000+00:00          1        18.50           72.11   
4    2025-02-12 14:26:46.023000+00:00          1        18.12           73.72   
..                                ...        ...          ...             ...   
501  2025-02-12 14:51:53.470000+00:00          1        19.29           55.85   
502  2025-02-12 14:51:56.469000+00:00          1        19.43           64.87   
503  2025-02-12 14:51:59.469000+00:00          1        19.73           55.13   
504  2025-02-12 14:52:02.469000+00:00          1        19.86           44.47   
505  2025-02-12 14:52:05.475000+00:00          1        20.05           42.55   

     batteryTemp  batteryVo

In [4]:
# Define stress condition (1 = Stress, 0 = Normal)
def detect_stress(row):
    if row["batteryCurrent"] > 400 or row["batteryTemp"] > 50 or row["stateOfCharge"] < 0.05 or row["batteryVoltage"] < 320:
        return 1  # Stress condition
    return 0  # Normal condition


# Apply stress detection
df["stressIndicator"] = df.apply(detect_stress, axis=1)

print(df)

                            timestamp  batteryId  ambientTemp  batteryCurrent  \
0    2025-02-12 14:26:34.190000+00:00          1        18.65           78.06   
1    2025-02-12 14:26:37.025000+00:00          1        18.36           81.42   
2    2025-02-12 14:26:40.020000+00:00          1        18.17           75.76   
3    2025-02-12 14:26:43.024000+00:00          1        18.50           72.11   
4    2025-02-12 14:26:46.023000+00:00          1        18.12           73.72   
..                                ...        ...          ...             ...   
501  2025-02-12 14:51:53.470000+00:00          1        19.29           55.85   
502  2025-02-12 14:51:56.469000+00:00          1        19.43           64.87   
503  2025-02-12 14:51:59.469000+00:00          1        19.73           55.13   
504  2025-02-12 14:52:02.469000+00:00          1        19.86           44.47   
505  2025-02-12 14:52:05.475000+00:00          1        20.05           42.55   

     batteryTemp  batteryVo

In [5]:
# Count the number of stress events
stress_count = df["stressIndicator"].sum()

print(f"Total stress events detected: {stress_count}")

Total stress events detected: 195


In [6]:
# Define Features and Target
features = ["stateOfCharge", "stateOfHealth", "batteryCurrent", "batteryVoltage", "kmh", "distance", "batteryTemp", "ambientTemp"]
X = df[features]
y = df["stressIndicator"]

In [7]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Define neural network
mlp_tf = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(len(set(y_train)), activation='softmax')  # Para clasificación multiclase
])

# Compile model
mlp_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2025-03-04 11:26:40.993254: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-03-04 11:26:40.993295: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: model-training-0
2025-03-04 11:26:40.993307: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: model-training-0
2025-03-04 11:26:40.993477: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.144.3
2025-03-04 11:26:40.993503: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.144.3
2025-03-04 11:26:40.993511: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.144.3


In [11]:
# Train model
mlp_tf.fit(X_train_scaled, y_train, epochs=10, batch_size=32)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7977 - loss: 0.5692 
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8460 - loss: 0.3818 
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8736 - loss: 0.2430 
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8724 - loss: 0.2047 
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9282 - loss: 0.1734 
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9758 - loss: 0.1250 
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9614 - loss: 0.1389 
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9727 - loss: 0.1108 
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9697 - loss: 0.1051 
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9795 - loss: 0.0808 


In [15]:
# Save model as keras and scaler for inference
mlp_tf.save("models/battery_stress_model.keras")
print("Model saved as TensorFlow format in 'models/battery_stress_model.keras'")

Model saved as TensorFlow format in 'models/battery_stress_model.keras'


In [18]:
# Load model
model = tf.keras.models.load_model("models/battery_stress_model.keras")

# Export SavedModel
tf.saved_model.save(model, "models/battery_stress_model")

print("Model converted to SavedModel format")

INFO:tensorflow:Assets written to: models/battery_stress_model/assets


INFO:tensorflow:Assets written to: models/battery_stress_model/assets


Model converted to SavedModel format


In [19]:
!mo --saved_model_dir models/battery_stress_model --output_dir openvino_model

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
2025-03-04 11:33:15.030502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741087995.057223   61398 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741087995.065379   61398 cuda_blas.cc:1418] Unable to register cuBLAS factory